In [4]:
import os
import re

import certifi
import pandas as pd
from openai import OpenAI

REQUESTS_CA_BUNDLE: c:\Users\Yi Jing\Documents\Y4S1\dsa4264\DSA4264-Detoxify\Lib\site-packages\certifi\cacert.pem
SSL_CERT_FILE: c:\Users\Yi Jing\Documents\Y4S1\dsa4264\DSA4264-Detoxify\Lib\site-packages\certifi\cacert.pem


In [ ]:
# Set the environment variables within your Python script
os.environ["REQUESTS_CA_BUNDLE"] = certifi.where()
os.environ["SSL_CERT_FILE"] = certifi.where()

print("REQUESTS_CA_BUNDLE:", os.environ.get("REQUESTS_CA_BUNDLE"))
print("SSL_CERT_FILE:", os.environ.get("SSL_CERT_FILE"))


client = OpenAI()

In [6]:
df = pd.read_csv("../combined_df.csv")

In [8]:
inc_crimes_hate = pd.read_csv("dfs_for_analysis/increase_crimes_hate_comments.csv")
inc_crimes_toxic = pd.read_csv("dfs_for_analysis/increase_crimes_toxic_comments.csv")
inc_gender_hate = pd.read_csv("dfs_for_analysis/increase_gender_hate_comments.csv")
inc_gender_toxic = pd.read_csv("dfs_for_analysis/increase_gender_toxic_comments.csv")
inc_race_hate = pd.read_csv("dfs_for_analysis/increase_race_hate_comments.csv")
inc_race_toxic = pd.read_csv("dfs_for_analysis/increase_race_toxic_comments.csv")

inc_crimes_hate["title_text"] = (
    inc_crimes_hate["Reddit Title"] + ": " + inc_crimes_hate["text"]
)

In [ ]:
inc_crimes_hate

,text,timestamp,username,link,link_id,parent_id,id,subreddit_id,moderation,year,word_count,Sensitive Group,Classification,clean_text,custom_topic,Topic_Words,Final Topic,Final Topic Name,Topic Num,Reddit Title,title_text
0,They're just hoping to kill one to silence many make an example of them.,5/4/2021 9:31,CSlv,/r/singapore/comments/mkf8jd/eight_people_convicted_for_selling_evaporisers/gtfpyhv/,t3_mkf8jd,t1_gtfjkvf,gtfpyhv,t5_2qh8c,"{'collapsed_reason': None, 'author_is_blocked': False, 'collapsed_reason_code': None, 'collapsed': False, 'controversiality': 0}",2021,14,No,Hate 3,theyre hoping kill one silence many make example,-1,"jail, jailed, arrest, detained",9.0,Crimes,2.0,eight people convicted for selling evaporisers,eight people convicted for selling evaporisers: They're just hoping to kill one to silence many make an example of them.
1,"to the boys and girls who tortured this boy, I wish you all very painful and violent deaths.",26/7/2022 16:45,MayhemBlankz,/r/singapore/comments/w8edrz/girl_who_joined_weeklong_group_assault_on_victim/ihqgqjw/,t3_w8edrz,t1_ihqglga,ihqgqjw,t5_2qh8c,"{'controversiality': 0, 'collapsed_reason_code': None, 'collapsed': False, 'collapsed_reason': None}",2022,18,Yes,Hate 3,boys girls tortured boy wish painful violent deaths,-1,"jail, jailed, arrest, detained",9.0,Crimes,2.0,girl who joined weeklong group assault on victim,"girl who joined weeklong group assault on victim: to the boys and girls who tortured this boy, I wish you all very painful and violent deaths."
2,"AGC: We'll murder your son, and then charge you with contempt for complaining about it!",27/4/2022 8:56,variably_random,/r/singapore/comments/ucuvsi/agc_flags_potential_contempt_over_false/i6dn1a5/,t3_ucuvsi,t3_ucuvsi,i6dn1a5,t5_2qh8c,"{'controversiality': 1, 'collapsed_reason_code': None, 'collapsed': False, 'collapsed_reason': None}",2022,15,No,Hate 3,agc well murder son charge contempt complaining,-1,"jail, jailed, arrest, detained",9.0,Crimes,2.0,agc flags potential contempt over false,"agc flags potential contempt over false: AGC: We'll murder your son, and then charge you with contempt for complaining about it!"
3,Someone looking to nail some helpless woman,28/3/2022 16:35,JoinTheRightClick,/r/singapore/comments/tppwdk/man_47_allegedly_molests_woman_in_lift_in_ghim/i2gelct/,t3_tppwdk,t1_i2dqq1h,i2gelct,t5_2qh8c,"{'controversiality': 0, 'collapsed_reason_code': None, 'collapsed': False, 'collapsed_reason': None}",2022,7,Yes,Hate 3,someone looking nail helpless woman,-1,"jail, jailed, arrest, detained",9.0,Crimes,2.0,man 47 allegedly molests woman in lift in ghim,man 47 allegedly molests woman in lift in ghim: Someone looking to nail some helpless woman
4,"Yes you deserve to have your neck snapped. Why Because if you had brought your mother to a doctor, he would have prescribed opioids. So your death would have been a result of negligence to do your homework or just out of sheer stupidity",20/6/2023 6:53,Bucafas,/r/SingaporeRaw/comments/14blzvg/if_i_had_500_grams_of_marijuana_but_its_for_my/joswtlc/,t3_14blzvg,t3_14blzvg,joswtlc,t5_xnx04,"{'banned_at_utc': None, 'mod_reason_by': None, 'mod_reason_title': None, 'banned_by': None, 'num_reports': 0, 'collapsed_reason_code': None, 'controversiality': 0, 'author_is_blocked': False, 'mod_note': None, 'collapsed': False, 'report_reasons': [], 'mod_reports': [], 'removal_reason': None, 'user_reports': [], 'collapsed_reason': None}",2023,44,No,Hate 3,yes deserve neck snapped brought mother doctor would prescribed opioids death would result negligence homework sheer stupidity,-1,"jail, jailed, arrest, detained",9.0,Crimes,2.0,if i had 500 grams of marijuana but its for my,"if i had 500 grams of marijuana but its for my: Yes you deserve to have your neck snapped. Why Because if you had brought your mother to a doctor, he would have prescribed opioids. So your death would have been a result of negligence to do your homework or just out of sheer stupidity"
5,Yes the compassion is disturb

In [ ]:
# Combine all rows into a single string
prompt_content = "".join(inc_crimes_hate["title_text"].tolist())

In [ ]:
# prompt_content = """Analyze the following topics and texts, and provide one main issue that emerges from the entire content:
# Topic: man_in_spore_misses_his_stop_holds_up_the_bus
# Text: It sounds like a meme but there's intense domestic propaganda in China about how overseas Chinese like us are.

# Topic: a_peer_review_of_todays_gen_y_speaks_opened_i_dont
# Text: Being a cis upper middle class Chinese Woman in Singapore is an even easier explanation than Garden variety.

# Topic: singapore_karen_tells_korean_twitch_streamer_to
# Text: Nothing shocking. This is how most Chinese boomers actually think.

# Topic: r/singapore_random_discussion_and_small_questions
# Text: I always wait to connect to a person then I say 'Taiwan number 1'. They always immediately hang up.

# Topic: any_idea_if_this_could_be_true
# Text: Why Taiwan if a free country, compared to China's dictatorship. Why would they falsify statistics, to gain what exactly."""

# Create the completion request
print(prompt_content)
response = client.chat.completions.create(
    model="gpt-4o-mini",  # Use the correct model identifier
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant. Identify one main topic that have caused the comments to be more hateful or toxic.",
        },
        {"role": "user", "content": prompt_content},
    ],
)

# Print the summarized response
print(response.choices[0].message.content)

In [ ]:
def analyze_toxicity(df_tuples, model_name="gpt-4o-mini"):
    responses = []
    for identifier, df in df_tuples:
        # Concatenate 'Reddit Title' and 'text' into 'title_text'
        df["title_text"] = df["Reddit Title"] + ": " + df["text"]
        # Join all entries into a single string
        prompt_content = " ".join(df["title_text"].tolist())

        # Call to your client's API to analyze the concatenated content individually
        response = client.chat.completions.create(
            model=model_name,  # Make sure to use the correct model identifier
            messages=[
                {
                    "role": "system",
                    "content": "You are a skilled assistant tasked with analyzing discussions. From the provided text, identify the principal issue that has led to increased hateful or toxic comments. Make it as succint as possible.",
                },
                {"role": "user", "content": prompt_content},
            ],
        )
        # Append the response content to the list along with the identifier
        responses.append((identifier, response.choices[0].message.content))

    return responses


# Load your datasets
datasets = {
    "Crimes Hate": pd.read_csv("dfs_for_analysis/increase_crimes_hate_comments.csv"),
    "Crimes Toxic": pd.read_csv("dfs_for_analysis/increase_crimes_toxic_comments.csv"),
    "Gender Hate": pd.read_csv("dfs_for_analysis/increase_gender_hate_comments.csv"),
    "Gender Toxic": pd.read_csv("dfs_for_analysis/increase_gender_toxic_comments.csv"),
    "Race Hate": pd.read_csv("dfs_for_analysis/increase_race_hate_comments.csv"),
    "Race Toxic": pd.read_csv("dfs_for_analysis/increase_race_toxic_comments.csv"),
}

# Create list of tuples for processing
df_tuples = [(identifier, df) for identifier, df in datasets.items()]

# Run analysis
analysis_results = analyze_toxicity(df_tuples)

# Output results with clear identification
for identifier, result in analysis_results:
    print(f"Results for {identifier}:\n{result}\n")

Results for Crimes Hate:
The principal issue leading to increased toxicity in these discussions is a pervasive desire for violent retribution against individuals involved in criminal activities, particularly those accused of sexual offenses. This sentiment fosters a toxic environment where harsh and aggressive punishments are not only suggested but celebrated, reflecting an extreme lack of empathy and a call for vigilante justice.

Results for Crimes Toxic:
The principal issue leading to increased toxicity in the provided text is the pervasive desire for violent, extreme punishments against individuals accused of various crimes, particularly against children. This reflects a toxic attitude towards justice and a lack of empathy for human life.

Results for Gender Hate:
The principal issue leading to increased toxicity in the discussion is the perpetuation of misogynistic attitudes and derogatory comments towards women, as highlighted by phrases such as "keep up that misogynistic mindset

In [ ]:
def analyze_toxicity(df_tuples, model_name="gpt-4o-mini"):
    responses = []
    for identifier, df in df_tuples:
        # Concatenate 'Reddit Title' and 'text' into 'title_text'
        df["title_text"] = df["Reddit Title"] + ": " + df["text"]
        # Join all entries into a single string
        prompt_content = " ".join(df["title_text"].tolist())

        # Call to your client's API to analyze the concatenated content individually
        response = client.chat.completions.create(
            model=model_name,  # Make sure to use the correct model identifier
            messages=[
                {
                    "role": "system",
                    "content": "You are a skilled assistant tasked with analyzing discussions. From the provided text, identify the principal issue that has led to increased hateful or toxic comments. Make it as succint as possible.",
                },
                {"role": "user", "content": prompt_content},
            ],
        )
        # Append the response content to the list along with the identifier
        responses.append((identifier, response.choices[0].message.content))

    return responses


# Load your datasets
datasets = {
    "Crimes Hate": pd.read_csv("dfs_for_analysis/increase_crimes_hate_comments.csv"),
    "Crimes Toxic": pd.read_csv("dfs_for_analysis/increase_crimes_toxic_comments.csv"),
    "Gender Hate": pd.read_csv("dfs_for_analysis/increase_gender_hate_comments.csv"),
    "Gender Toxic": pd.read_csv("dfs_for_analysis/increase_gender_toxic_comments.csv"),
    "Race Hate": pd.read_csv("dfs_for_analysis/increase_race_hate_comments.csv"),
    "Race Toxic": pd.read_csv("dfs_for_analysis/increase_race_toxic_comments.csv"),
}

# Create list of tuples for processing
df_tuples = [(identifier, df) for identifier, df in datasets.items()]

# Run analysis
analysis_results = analyze_toxicity(df_tuples)

# Output results with clear identification
for identifier, result in analysis_results:
    print(f"Results for {identifier}:\n{result}\n")

Results for Crimes Hate:
The principal issue leading to increased hateful or toxic comments in the discussions is the public's intense emotional response to cases of violence, sexual offenses, and perceived injustices within the legal system, prompting extreme calls for punishment, including death or severe retribution.

Results for Crimes Toxic:
The principal issue leading to increased hateful or toxic comments in the discussions is the prevalence of sexual violence and abuse against children, which provokes extreme and violent reactions from commentators.

Results for Gender Hate:
The principal issue leading to increased hateful or toxic comments in the discussion is the contentious debate surrounding gender equality, particularly in relation to societal roles and expectations for men and women, including topics like national service, dating standards, and systemic oppression. This has sparked a polarized environment where extreme views on gender dynamics and misogyny are prevalent.
